In [1]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
#Primary Source contenct
main_url = 'https://awesomeatyourjob.com/podcast/'
req = Request(main_url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
main_soup = BeautifulSoup(webpage, 'html.parser')

In [3]:
#Generate the urls for all of the links on the page
scrape_list = []
for link in main_soup.find_all("h2", class_="title"):
    scrape_list.append(link.find("a")["href"])
    
#Create empty dataframe
scrape_data = pd.DataFrame(columns=["title", "text", "date"])

#Printing the list to make sure I got the outcome I wanted
print(scrape_list)

['https://awesomeatyourjob.com/835-how-to-thrive-amid-stress-and-irritation-with-dr-sharon-melnick/', 'https://awesomeatyourjob.com/834-how-to-end-micromanagement-once-and-for-all-with-lia-garvin/', 'https://awesomeatyourjob.com/833-the-four-step-process-to-influencing-people-and-decisions-with-andres-lares/', 'https://awesomeatyourjob.com/832-how-to-restore-yourself-from-burnout-with-christina-maslach/', 'https://awesomeatyourjob.com/831-how-to-manage-multiple-projects-without-the-overwhelm-with-elizabeth-harrin/']


In [4]:
# the loop visits each url in the scrape_list and then collects the text content from each page, creating a new list
for i in range(len(scrape_list)):
    new_url = scrape_list[i]
    req = Request(new_url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, 'html.parser')
    
    #Collects the title content from the page
    title_add = soup.find("title").get_text()
    
    #Collects the transcript from the page
    transcripts = soup.find_all('div', {'class': 'wpb_wrapper'})
    transcript = transcripts[-1]
    text_add = transcript.get_text()
    
    #Collects the date from the page
    date_add = soup.select_one('span[class="meta-date date updated"]').text
    
    #Append the data to the dataframe
    new_data = pd.DataFrame({"title": [title_add], "text": [text_add], "date": [date_add]})
    scrape_data = pd.concat([scrape_data, new_data], ignore_index=True)

    #Add 3 second delay
    time.sleep(3) 

In [5]:
# find the first div with class "next"
next_div = main_soup.find("div", class_="next")

# find the first anchor tag within that div
next_link = next_div.find("a")

# get the href attribute of that anchor tag
page_url = next_link["href"]

#Printing the next page to test
print(page_url)

https://awesomeatyourjob.com/podcast/page/2/


In [6]:
while True:
    req = Request(page_url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    p_soup = BeautifulSoup(webpage, 'html.parser')
    
    #Generate the urls for all of the links on the page
    page_list = []
    for link in p_soup.find_all("h2", class_="title"):
        page_list.append(link.find("a")["href"])
        
    # the loop visits each url in the scrape_list and then collects the text content from each page, creating a new list
    for i in range(len(page_list)):
        new_url = page_list[i]
        #Print the page for troubleshooting
        print(new_url)
        req = Request(new_url, headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        soup = BeautifulSoup(webpage, 'html.parser')

        #Collects the title content from the page
        title_add = soup.find("title").get_text()

        #Collects the transcript from the page
        transcripts = soup.find_all('div', {'class': 'wpb_wrapper'})
        #not all pages have a transcript
        try:
            transcript = transcripts[-1]
        except IndexError:
            continue
        text_add = transcript.get_text()

        #Collects the date from the page
        date_add_add = soup.select_one('span[class="meta-date date updated"]').text

        #Append the data to the dataframe
        new_data = pd.DataFrame({"title": [title_add], "text": [text_add], "date": [date_add]})
        scrape_data = pd.concat([scrape_data, new_data], ignore_index=True)

        #Add 3 second delay
        time.sleep(3)
        
    #Generate the next page
    # find the first div with class "next"
    next_div = p_soup.find("div", class_="next")

    # find the first anchor tag within that div
    next_link = next_div.find("a")
    if next_link:
         # get the href attribute of that anchor tag
        page_url = next_link["href"]
    else:
        break        
    
    #printing the next page to make sure everything is working right
    print(page_url)

https://awesomeatyourjob.com/830-lessons-learned-from-the-worlds-longest-scientific-study-on-happiness-with-robert-waldinger/
https://awesomeatyourjob.com/829-how-to-write-so-people-will-read-with-casey-mank/
https://awesomeatyourjob.com/828-how-to-reach-your-epic-goals-and-unlock-elite-performance-with-bryan-gillette/
https://awesomeatyourjob.com/how-to-make-the-most-of-conflict-with-liane-davey/
https://awesomeatyourjob.com/826-finding-calm-in-an-uncertain-and-stressful-world-with-jacqueline-brassey/
https://awesomeatyourjob.com/podcast/page/3/
https://awesomeatyourjob.com/825-the-six-steps-of-masterful-delegation-with-aaron-schmookler/
https://awesomeatyourjob.com/824-thriving-amid-information-overload-with-ross-dawson/
https://awesomeatyourjob.com/823-how-to-collaborate-smarter-with-dr-heidi-gardner/
https://awesomeatyourjob.com/822-how-to-take-your-next-best-step-when-life-is-uncertain-with-jeff-henderson/
https://awesomeatyourjob.com/821-how-to-keep-calm-and-defuse-tensions-in-co

In [7]:
#Save the dataframe

scrape_data.to_csv("raw.csv")